In [0]:
%pip install --upgrade openaI numpy==1.23.5 databricks-vectorsearch

In [0]:
dbutils.library.restartPython()

In [0]:
import json
import pandas as pd
from openai import OpenAI
from openai import AssistantEventHandler, AzureOpenAI

In [0]:
%run ../common/nb_init

#Fragen

  {
    "id": "social_01_01",
    "table": "consolidated_socials",
    "query": "Wie viele Beiträge wurden im Januar 2025 auf Instagram veröffentlicht?",
    "expected": ""
  }, 

  -> Bei Alternative letzte Stelle hochzählen

    {
    "id": "social_01_02",
    "table": "consolidated_socials",
    "query": "Wie viele Beiträge wurden im Januar 2025 auf Instagram veröffentlicht?",
    "expected": ""
  },

In [0]:
testcases = [
  {
    "id": "social_01_01",
    "table": "consolidated_socials",
    "query": "Wie viele Beiträge wurden im Januar 2025 auf Instagram veröffentlicht?",
    "expected": ""
  },
  {
    "id": "social_02_01",
    "table": "consolidated_socials",
    "query": "Welcher Channel hatte im Februar 2025 die meisten Impressions?",
    "expected": ""
  },
  {
    "id": "social_03_01",
    "table": "consolidated_socials",
    "query": "Gib den Durchschnitt der EngagementRateInPercent für alle LinkedIn-Posts im Q1 2025 an.",
    "expected": ""
  },
  {
    "id": "social_04_01",
    "table": "consolidated_socials",
    "query": "Zeige die Top 5 Posts mit der höchsten EngagementRateInPercent im März 2025.",
    "expected": ""
  },
  {
    "id": "social_05_01",
    "table": "consolidated_socials",
    "query": "Welcher PostType erzielte im April 2025 die meisten Impressions?",
    "expected": ""
  },
  {
    "id": "social_06_01",
    "table": "consolidated_socials",
    "query": "Vergleiche die Impressions von Facebook-Posts und Instagram-Posts im Mai 2025.",
    "expected": ""
  },
  {
    "id": "social_07_01",
    "table": "consolidated_socials",
    "query": "Wie viele Posts sind mehreren Strategischen Themen zugeordnet?",
    "expected": ""
  },
  {
    "id": "social_08_01",
    "table": "consolidated_socials",
    "query": "Welche Themenbereiche hatten die höchste durchschnittliche EngagementRateInPercent im ersten Halbjahr 2025?",
    "expected": ""
  },
  {
    "id": "social_09_01",
    "table": "consolidated_socials",
    "query": "Gibt es einen Monat, in dem der Channel X neue Inhalte veröffentlicht hat?",
    "expected": ""
  },
  {
    "id": "social_10_01",
    "table": "consolidated_socials",
    "query": "Welcher Post hatte die meisten Impressions insgesamt?",
    "expected": ""
  },
  {
    "id": "eco_01_01",
    "table": "consolidated_eco_journal",
    "query": "Wie viele Artikel wurden im Januar 2025 veröffentlicht?",
    "expected": ""
  },
  {
    "id": "eco_02_01",
    "table": "consolidated_eco_journal",
    "query": "Welcher Artikel hatte die meisten Views im Februar 2025?",
    "expected": ""
  },
  {
    "id": "eco_03_01",
    "table": "consolidated_eco_journal",
    "query": "Berechne die durchschnittliche Bounce_rate im Q1 2025.",
    "expected": ""
  },
  {
    "id": "eco_04_01",
    "table": "consolidated_eco_journal",
    "query": "Vergleiche die Gesamtzahl der Sessions im März und April 2025.",
    "expected": ""
  },
  {
    "id": "eco_05_01",
    "table": "consolidated_eco_journal",
    "query": "Welcher Themenbereich hat die höchste durchschnittliche User_engagement-Zeit?",
    "expected": ""
  },
  {
    "id": "rep_01_01",
    "table": "reputationsindex",
    "query": "Wie entwickelt sich die Bekanntheit_in_Prozent von EnBW über die Monate 2025?",
    "expected": ""
  },
  {
    "id": "rep_0_01",
    "table": "reputationsindex",
    "query": "Welches Unternehmen hatte im März 2025 den höchsten Bekanntsheitwert?",
    "expected": ""
  },
  {
    "id": "rep_03_01",
    "table": "reputationsindex",
    "query": "Vergleiche den durchschnittlichen Bekanntheitswert von RWE, EnBW und E.ON im Q1 2025.",
    "expected": ""
  },
  {
    "id": "rep_04_01",
    "table": "reputationsindex",
    "query": "Gibt es Monate, in denen alle drei Unternehmen denselben Wert hatten?",
    "expected": ""
  },
  {
    "id": "rep_05_01",
    "table": "reputationsindex",
    "query": "Welches Unternehmen zeigt die größte Schwankung in der Bekanntheit_in_Prozent im ersten Halbjahr 2025?",
    "expected": ""
  }
]


#Evaluierung

In [0]:
secret= get_secret("openai-assistant-key")
endpoint = get_secret("openai-assistant-endpoint")
ASSISTANT_ID = get_secret("openai-assistant-id")

In [0]:
# Client
client = AzureOpenAI(
    api_key=secret,
    api_version="2024-05-01-preview",
    azure_endpoint=endpoint
)

# Funktion: Assistant-Call
def ask_bot(query):
    # neuen Thread mit User-Eingabe anlegen
    thread = client.beta.threads.create(
        messages=[
            {"role": "user", "content": query}
        ]
    )

    # Thread erstellen
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=ASSISTANT_ID
    )

    # Nachrichten auslesen
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    answer = messages.data[0].content[0].text.value

    return answer.strip()

# Funktion: LLM-Judge
def llm_judge(query, answer, expected):
    if not expected:  # keine GT → kein Scoring
        return None
    
    judge_prompt = f"""
        Du bist ein Evaluator für Chatbot-Antworten.
        Bewerte die Qualität einer Antwort im Vergleich zur erwarteten Lösung.

        Frage: {query}
        Antwort des Bots: {answer}
        Erwartet: {expected}

        Bewerte von 1 bis 5:
        1 = falsch oder unverständlich
        3 = teilweise korrekt, aber unvollständig oder unklar
        5 = vollständig korrekt und klar formuliert

        Gib NUR die Zahl zurück.
    """

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": judge_prompt}]
    )
    try:
        return int(resp.choices[0].message.content.strip())
    except:
        return None

# Test-Loop
results = []
for case in testcases:
    query = case["query"]
    expected = case["expected"]

    # Bot-Antwort
    answer = ask_bot(query)

    # Bewertung (nur falls Expected eingetragen ist)
    score = llm_judge(query, answer, expected) if expected else None

    results.append({
        "id": case["id"],
        "table": case["table"],
        "query": query,
        "expected": expected,
        "answer": answer,
        "score": score
    })

# DataFrame bauen
df = pd.DataFrame(results)

#Anzeigen der Ergebnisse

In [0]:
df.display()